You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [1]:
%pip install -q -r requirements.txt
# !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
# %cd Vessel-Geometric-Transformers
!pre-commit autoupdate
!pre-commit install

ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
Note: you may need to restart the kernel to use updated packages.
[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


In [2]:
from src.utils import load_config
import os
from config import DatasetConfig

config = load_config(os.path.join("config/config.yaml"))
dataset_config: DatasetConfig = config.dataset

In [3]:
from src.data.dataset import VesselDataset

dataset = VesselDataset(dataset_config, "train") # type: ignore
print(dataset.raw_file_names)
print(dataset.processed_file_names)

['/home/neverorfrog/code/deep-learning/gatr/dataset/stead']
['train/data.pt']
